# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchsummary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Set Hyper Parameters

In [3]:
hyper_parameters = {
    "SEED":20180724,
    "notebookName":"ComplexModel2",
    "nepochs":10000,
    "batch_size":256,
    "learning_rate":0.001,
    "window_size":128,
    "max_patience_count":1000
}

In [4]:
torch.manual_seed(hyper_parameters["SEED"])

In [5]:
f"./models/{hyper_parameters['notebookName']}"

'./models/ComplexModel2'

In [6]:

PATH = Path(f"./models/{hyper_parameters['notebookName']}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files = 2

# Set Path

In [7]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")

# Help Functions

# Load Data

In [8]:
df_train_default = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))

In [9]:
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [10]:
for col in df_train_default.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
elapsedRealtimeNanos
yawDeg
rollDeg
pitchDeg
utcTimeMillis_Status
SignalCount
SignalIndex
ConstellationType
Svid
CarrierFrequencyHz
Cn0DbHz
AzimuthDegrees
ElevationDegrees
UsedInFix
HasAlmanacData
HasEphemerisData
BasebandCn0DbHz
utcTimeMillis_UncalMag
elapsedRealtimeNanos_UncalMag
UncalMagXMicroT
UncalMagYMicroT
UncalMagZMicroT
BiasXMicroT
BiasYMicroT
BiasZMicroT
utcTimeMillis_UncalAccel
elapsedRealtimeNanos_UncalAccel
UncalAccelXMps2
UncalAccelYMps2
UncalAccelZMps2
BiasXMps2
BiasYMps2
BiasZMps2
utcTimeMillis_UncalGyro
elapsedRealtimeNanos_UncalGyro
UncalGyroXRadPerSec
UncalGyroYRadPerSec
U

# Dataloader

In [11]:
df_train_default['phone'].value_counts()

2021-04-22-US-SJC-1_Pixel4             2890
2021-04-22-US-SJC-1_SamsungS20Ultra    2826
2020-09-04-US-SF-2_Mi8                 2500
2021-04-29-US-SJC-2_SamsungS20Ultra    2370
2020-09-04-US-SF-2_Pixel4              2349
                                       ... 
2021-01-05-US-SVL-2_Pixel4XL           1193
2020-06-05-US-MTV-1_Pixel4XLModded     1123
2021-04-26-US-SVL-1_Mi8                1036
2021-04-26-US-SVL-1_Pixel5             1034
2020-05-14-US-MTV-2_Pixel4XLModded      577
Name: phone, Length: 73, dtype: int64

# Split functions

In [12]:
def CustomTrainValidSplit(df:pd.DataFrame, valid_size):
    phones = df['phone'].unique()
    
    valid_num = int(len(phones) * valid_size)
    train_num = len(phones) - valid_num
    
    indexes = np.array(range(len(phones)))
    indexes = np.random.choice(indexes, len(indexes), replace = False)
    
    df_train = []
    for phone in phones[indexes[:train_num]]:
        df_train.append(df[df['phone'] == phone])
    df_train = pd.concat(df_train).reset_index().drop(columns = 'index')
    
    df_valid = []
    for phone in phones[indexes[train_num:-1]]:
        df_valid.append(df[df['phone'] == phone])
    df_valid = pd.concat(df_valid).reset_index().drop(columns = 'index')
    
    return df_train, df_valid
    
df_train, df_valid = CustomTrainValidSplit(df_train_default.sort_values(by = ['phone', 'millisSinceGpsEpoch'], ignore_index = True), valid_size = 0.1)
df_test = df_test.sort_values(by = ['phone', 'millisSinceGpsEpoch'], ignore_index = True)
print(df_train.shape, df_valid.shape)
    

(118266, 148) (11474, 148)


In [13]:
df_train

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,...,GPS_L1,GPS_L5,GAL_E1,GAL_E5A,GLO_G1,BDS_B1I,BDS_B1C,BDS_B2A,QZS_J1,QZS_J5
0,2020-07-17-US-MTV-1,Mi8,1279059935000,37.428288,-122.072583,-16.14,2020-07-17-US-MTV-1_Mi8,669.00,0.8,0.0,...,0,0,1,0,0,0,0,0,0,0
1,2020-07-17-US-MTV-1,Mi8,1279059936000,37.428271,-122.072521,-24.96,2020-07-17-US-MTV-1_Mi8,670.00,0.8,0.0,...,1,0,0,0,0,0,0,0,0,0
2,2020-07-17-US-MTV-1,Mi8,1279059937000,37.428234,-122.072572,-24.38,2020-07-17-US-MTV-1_Mi8,671.00,0.8,0.0,...,1,0,0,0,0,0,0,0,0,0
3,2020-07-17-US-MTV-1,Mi8,1279059938000,37.428260,-122.072544,-21.39,2020-07-17-US-MTV-1_Mi8,672.00,0.8,0.0,...,1,0,0,0,0,0,0,0,0,0
4,2020-07-17-US-MTV-1,Mi8,1279059939000,37.428266,-122.072531,-30.24,2020-07-17-US-MTV-1_Mi8,673.00,0.8,0.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118261,2021-04-15-US-MTV-1,Pixel4,1302558854445,37.424370,-122.092000,-34.55,2021-04-15-US-MTV-1_Pixel4,1873.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
118262,2021-04-15-US-MTV-1,Pixel4,1302558855445,37.424383,-122.091978,-37.73,2021-04-15-US-MTV-1_Pixel4,1874.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
118263,2021-04-15-US-MTV-1,Pixel4,1302558856445,37.424386,-122.091979,-36.06,2021-04-15-US-MTV-1_Pixel4,1875.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
118264,2021-04-15-US-MTV-1,Pixel4,1302558857445,37.424380,-122.091974,-38.94,2021-04-15-US-MTV-1_Pixel4,1876.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0


# Create Data Loader

## Moving Window Function

In [14]:
def MovingWindow(current_index, max_index, window_size, ratio = 0.5):
    left_window = int(window_size * ratio)
    right_window = window_size - left_window
    
    left_index = np.arange(current_index - left_window, current_index)
    right_index = np.arange(current_index, current_index + right_window)
    
    index = np.concatenate([left_index, right_index])
    index[index < 0] = 0
    index[index >= max_index] = max_index-1
    
    return index

## Custom Dataset

In [15]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.DataFrame, 
                 features = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM'], 
                 labels = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM'],
                 window_size = 100,
                 train = False,
                 augment = False,
                 crop_ratio = 0.0,
                device = 'cpu'):
        self.df = df
        self.features = features
        self.labels = labels
        self.len = df.shape[0]
        self.window_size = window_size
        self.train = train
        self.augment = augment
        self.crop_ratio = crop_ratio
        self.device = device
        self.phone = self.df['phone'].values
        self.millisSinceGpsEpoch = self.df['millisSinceGpsEpoch'].values
        
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        origin_idx = idx
        phone = self.phone[idx]
        df = self.df
        df = df[df['phone'] == phone]
        
        start_index = df.index[0]
        
        self.data = df[features].astype(float).values
        if self.train == True:
            self.true = df[labels].astype(float).values
        else:
            self.true = []
        
        window_index = MovingWindow(idx - start_index, df.shape[0], self.window_size)
        data = self.data[window_index, :]
        
            
        indx = [self.phone[idx], self.millisSinceGpsEpoch[idx]]
        
        # data shape : num_of_features X window_size
        # true shape : num_of_labels X 1
        data = torch.Tensor(data)
        if self.augment:
            crop_size = int(self.crop_ratio * self.window_size * np.random.rand(1))
            crop_start_index = np.random.randint(0, self.window_size - crop_size)
            data[crop_start_index:crop_size, :] = 0
        
        if self.train is False:
            true = []
        else:
            true = self.true[idx - start_index, :]
            true = torch.Tensor(true.astype(float))
        return data, true, indx
    


## Feature Select

In [16]:
features = [
    'latDeg', 
    'lngDeg', 
    'heightAboveWgs84EllipsoidM',
    'dlatDeg_Scaled',
    'dlngDeg_Scaled',
    'dheight_Scaled',
    'xSatPosM_Scaled',
    'ySatPosM_Scaled',
    'zSatPosM_Scaled',
    'xSatVelMps_Scaled',
    'ySatVelMps_Scaled',
    'zSatVelMps_Scaled',
    'UncalGyroXRadPerSec_Scaled',
    'UncalGyroYRadPerSec_Scaled',
    'UncalGyroZRadPerSec_Scaled',
    'DriftXRadPerSec_Scaled',
    'DriftYRadPerSec_Scaled',
    'DriftZRadPerSec_Scaled',
    'UncalAccelXMps2_Scaled',
    'UncalAccelYMps2_Scaled',
    'UncalAccelZMps2_Scaled',
    'BiasXMps2_Scaled',
    'BiasYMps2_Scaled',
    'BiasZMps2_Scaled',
    'UncalMagXMicroT_Scaled',
    'UncalMagYMicroT_Scaled',
    'UncalMagZMicroT_Scaled',
    'BiasXMicroT_Scaled',
    'BiasYMicroT_Scaled',
    'BiasZMicroT_Scaled',
    'yawDeg_Scaled',
    'rollDeg_Scaled',
    'pitchDeg_Scaled',
    'GPS_L1', 
    'GPS_L5', 
    'GAL_E1', 
    'GAL_E5A', 
    'GLO_G1', 
    'BDS_B1I', 
    'BDS_B1C', 
    'BDS_B2A', 
    'QZS_J1', 
    'QZS_J5'
]
status_features = [
]

labels = [
    't_latDeg', 
    't_lngDeg', 
    't_heightAboveWgs84EllipsoidM',
#     'courseDegree',
#     'hDop',
#     'vDop',
#     'speedMps'
         ]

print(df_train[features].shape)
print(df_train[features].describe())


(118266, 43)
              latDeg         lngDeg  heightAboveWgs84EllipsoidM  \
count  118266.000000  118266.000000               118266.000000   
mean       37.434598    -122.151897                   23.565345   
std         0.084241       0.149561                   73.314982   
min        37.321683    -122.472214                -6157.470000   
25%        37.368133    -122.275107                  -26.750000   
50%        37.424295    -122.116574                    3.590000   
75%        37.469090    -122.067592                   58.690000   
max        37.690836    -121.881855                13701.980000   

       dlatDeg_Scaled  dlngDeg_Scaled  dheight_Scaled  xSatPosM_Scaled  \
count   118266.000000   118266.000000   118266.000000    118266.000000   
mean        -0.021044        0.020760        0.005305        -0.000129   
std          1.008086        0.998718        1.003252         0.719928   
min         -3.367212       -2.837517      -84.852009        -1.703958   
25%         -

## Get Datasets

In [17]:
train_data = CustomDataset(df_train, 
                           features = features, labels = labels, 
                           window_size = hyper_parameters['window_size'], 
                           train = True, 
                           augment = True,
                           crop_ratio = 0.1,
                           device = device)
valid_data = CustomDataset(df_valid, 
                           features = features, labels = labels, 
                           window_size = hyper_parameters['window_size'], 
                           train = True, 
                           augment = False,
                           crop_ratio = 0.0,
                           device = device)
test_data = CustomDataset(df_test, 
                        features = features, labels = labels, 
                        window_size = hyper_parameters['window_size'], 
                        train = False, 
                        augment = False,
                        crop_ratio = 0.0,
                        device = device)

## Get Data Loader

In [18]:
train_loader = DataLoader(train_data, batch_size = hyper_parameters['batch_size'], shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = hyper_parameters['batch_size'], shuffle = False)
test_loader = DataLoader(test_data, batch_size = hyper_parameters['batch_size'], shuffle = False)

# Build Model
## Define Loss and Score

In [19]:
class GPSLossScore(nn.Module):
    def __init__(self):
        super().__init__()
        self.EARTH_RADIUS = 6_367_000
        self.loss = nn.SmoothL1Loss()
        pass
    
    def forward(self, predict:torch.Tensor, target:torch.Tensor):
        loss = self.loss(predict[:,:2], target[:,:2])
        if (loss <1e-4):
            loss = self.gps_loss(predict, target)
        return loss
    
    def torch_haversine(self,lat1, lon1, lat2, lon2):
    
        lat1=lat1 % 360
        lon1=lon1 % 360
        lat2=lat2 % 360
        lon2=lon2 % 360

        lat1, lat2, lon1, lon2 = map(torch.deg2rad, [lat1, lat2, lon1, lon2])

        dlat = (lat2 - lat1)
        dlon = (lon2 - lon1)

        a = torch.sin(dlat / 2.0)**2 + torch.cos(lat1) * torch.cos(lat2) * (torch.sin(dlon / 2.0)**2)
        c = 2 * torch.arcsin(a ** 0.5)

        dist = self.EARTH_RADIUS * c

        return dist
    
    def custom_loss(self, predict:torch.Tensor, target:torch.Tensor):
        latlng_loss = self.loss(predict[:,:2], target[:,:2])
        other_loss = self.loss(predict[:,2:], target[:,2:])
        loss = 0.9 * latlng_loss + 0.1 * other_loss
        
        return loss, latlng_loss, other_loss

    def gps_loss(self, predict:torch.Tensor, target:torch.Tensor):
        dist = self.torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])

        loss = dist.mean()

        return loss

    def gps_score(self, predict:torch.Tensor, target:torch.Tensor):
        dist = self.torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])

        score = (torch.quantile(dist, 0.5) + torch.quantile(dist, 0.95))/2

        return score
    

## Build Custom Model

In [20]:

class ConvBlock(nn.Module):
    def __init__(self, input_features, features = 128):
        super().__init__()
        
        self.fuse = nn.Conv1d(input_features + features, features, kernel_size = 1)
        self.conv = nn.Conv1d(input_features, features, kernel_size=3, padding = 1)
        self.batch = nn.BatchNorm1d(features)
        self.pool = nn.AvgPool1d(kernel_size = 2)
        
    def forward(self, x):
        skip = x
        
        x = self.conv(x)
        x = F.relu(x)
        x = self.batch(x)
        x = torch.cat([skip, x], axis = 1)
        x = self.fuse(x)
        x = self.pool(x)
        return x
    
class BaseModel(nn.Module):
    def __init__(self, input_size = (100, 3), output_size = 3):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.conv1 = ConvBlock(input_size[1], 256)
        self.conv2 = ConvBlock(256, 512)
        
        self.batch_norm = nn.BatchNorm1d(512)
        self.layer_norm = nn.LayerNorm([32, 512])
        
        self.fc1 = nn.Linear(32*512, 2048)
        self.layer_norm1 = nn.LayerNorm(2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.layer_norm2 = nn.LayerNorm(1024)
        self.fc3 = nn.Linear(1024, 512)
        self.layer_norm3 = nn.LayerNorm(512)
        self.fc4 = nn.Linear(512, output_size)
        self.layer_norm4 = nn.LayerNorm(output_size)
        
        self.drop06 = nn.Dropout(0.6)
        self.drop03 = nn.Dropout(0.3)
        self.drop01 = nn.Dropout(0.1)
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.kaiming_normal_(self.fc3.weight)
        nn.init.kaiming_normal_(self.fc4.weight)
        
        
    def forward(self, x):
        input_size = self.input_size 
        output_size = self.output_size
        
        x[:,:,:2] = torch.deg2rad(x[:,:,:2])
        
        
        x = x.transpose(2,1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.transpose(2,1)
        
        x = x.reshape(-1, 32*512)
        
        x = self.drop06(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.layer_norm1(x)
        
        x = self.drop06(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.layer_norm2(x)
        
        x = self.drop03(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.layer_norm3(x)
        
        x = self.drop01(x)
        x = self.fc4(x)
        x = self.layer_norm4(x)
        
        x[:,:2] = torch.rad2deg(x[:,:2])
        
        return x

## Compile Model

In [21]:
model = BaseModel((hyper_parameters['window_size'], len(features)), len(labels))
model.to(device)
# model.load_state_dict(torch.load("./models/Baseline3/model-4.pth"))

# loss_func = nn.SmoothL1Loss()
loss_func = GPSLossScore()
optimizer = optim.Adam(model.parameters(), lr = hyper_parameters['learning_rate'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                mode = 'min',
                                                factor = 0.1,
                                                patience = 5,
                                                verbose = True)

# Fit and Validate
## Train

In [22]:
def train(epoch, progress_log, status_log):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    for data, targets, _ in progress_log:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(outputs)
        ground.append(targets)
        
        status_log.set_description_str(
            f"train status: loss - {loss_func(outputs, targets)} | {nn.SmoothL1Loss()(outputs, targets)}, score - {loss_func.gps_score(outputs, targets)}")

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = loss_func.gps_score(predict, ground)
    return loss, meas

## Valid

In [23]:
def valid(progress_log, status_log):
    model.eval()  # 신경망을 추론 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    predict = []
    ground = []
    
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets, _ in progress_log:
            
            data = data.to(device)
            targets = targets.to(device)
            
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
            
            predict.append(outputs)
            ground.append(targets)
            
        status_log.set_description_str(f"valid status: loss - {loss_func(outputs, targets)} | {nn.SmoothL1Loss()(outputs, targets)}, score - {loss_func.gps_score(outputs, targets)}")
    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = loss_func.gps_score(predict, ground)
    return loss, meas

## Test

In [24]:
def test(dataloader):
    model.eval()  # 신경망을 추론 모드로 전환
    
    output_list = []
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, _, index in tqdm(dataloader):
            data = data.to(device)
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            df_temp = pd.DataFrame()
            df_temp['phone'] = index[0]
            df_temp['millisSinceGpsEpoch'] = index[1]
            df_temp[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']] = outputs[:,:3].to('cpu').numpy()
            output_list.append(df_temp)
    
    predicts = pd.concat(output_list)
    return predicts
            

## Fit

In [ ]:
train_loss_list = []
train_meas_list = []
valid_loss_list = []
valid_meas_list = []

patience_count = 0
min_valid_meas = np.inf
checkpoint_name = ""

epoch_prog = tqdm(range(hyper_parameters['nepochs']), position = 0, desc = "EPOCH")
epoch_status_log = tqdm(total=0, position = 2, bar_format='{desc}')
train_status_log = tqdm(total=0, position = 3, bar_format='{desc}')
valid_status_log = tqdm(total=0, position = 4, bar_format='{desc}')

if not os.path.isdir(f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/"):
    os.mkdir(f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/")
    
for epoch in epoch_prog:
    train_prog = tqdm(train_loader, position = 5, desc = 'Train', leave = False)
    valid_prog = tqdm(valid_loader, position = 6, desc = 'Valid', leave = False)

    train_loss, train_meas = train(epoch, train_prog, train_status_log)
    valid_loss, valid_meas = valid(valid_prog, valid_status_log)
    
    scheduler.step(valid_meas)
    if valid_meas < min_valid_meas:
        min_valid_meas = valid_meas
        checkpoint_name = f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/model-{epoch}-{min_valid_meas}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > hyper_parameters['max_patience_count']):
            break
    
    train_loss_list.append(train_loss)
    train_meas_list.append(train_meas)
    valid_loss_list.append(valid_loss)
    valid_meas_list.append(valid_meas)
    
    status = f"EPOCH:{epoch}/{hyper_parameters['nepochs']} train status: loss - {train_loss}, score - {train_meas}, valid status: loss - {valid_loss}, score - {valid_meas}"
    epoch_status_log.set_description_str(status)

    print(status)

history = dict()
history['train_loss'] = train_loss_list
history['train_meas'] = train_meas_list
history['valid_loss'] = valid_loss_list
history['valid_meas'] = valid_meas_list


EPOCH:   0%|          | 0/10000 [00:00<?, ?it/s]

Train:   0%|          | 0/462 [00:00<?, ?it/s]

Valid:   0%|          | 0/45 [00:00<?, ?it/s]

EPOCH:0/10000 train status: loss - 6.988936424255371, score - 2068186.75, valid status: loss - 0.004245317541062832, score - 16168.9033203125


Train:   0%|          | 0/462 [00:00<?, ?it/s]

...

Train:   0%|          | 0/462 [00:00<?, ?it/s]

Valid:   0%|          | 0/45 [00:00<?, ?it/s]

EPOCH:25/10000 train status: loss - 0.0067878481931984425, score - 18954.6484375, valid status: loss - 0.003960108384490013, score - 15661.9013671875


Train:   0%|          | 0/462 [00:00<?, ?it/s]

Valid:   0%|          | 0/45 [00:00<?, ?it/s]

## Visualization

In [ ]:
plt.figure(figsize = (16,6))
plt.subplot(2,1,1)
plt.plot(history['train_loss'], label = 'train')
plt.plot(history['valid_loss'], label = 'valid')
plt.ylabel('loss')

plt.subplot(2,1,2)
plt.plot(history['train_meas'], label = 'train')
plt.plot(history['valid_meas'], label = 'valid')
plt.ylabel('meas')

#  Output

In [ ]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)
submission.head()

In [ ]:
model.load_state_dict(torch.load(checkpoint_name))
torch.save(model.state_dict(), f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/model-{epoch}-{min_valid_meas}.pth")

In [ ]:
predict = test(test_loader)
print(predict.shape)
predict.head()

In [ ]:
submission = submission[['phone', 'millisSinceGpsEpoch']].merge(predict[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']]
                                                                , on = ['phone', 'millisSinceGpsEpoch'])
print(submission.shape)
submission.head()

In [ ]:
submission.to_csv(f"./models/{hyper_parameters['notebookName']}/result-{num_files}.csv", index = False)
pd.DataFrame([]).to_csv(PATH / f"{num_files}")